### Setup some basic stuff

In [1]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [2]:
import folium
import folium.features as fof
import folium.utilities as ful
import branca.element as bre
import json
import geojson as gj
import arrow

import shapely.geometry as shpg
import pandas as pd
import geopandas as gpd

In [3]:
def lonlat_swap(lon_lat):
    return list(reversed(lon_lat))

In [4]:
def get_row_count(n_maps, cols):
    rows = (n_maps / cols)
    if (n_maps % cols != 0):
        rows = rows + 1
    return rows

In [5]:
def get_one_marker(loc, disp_color):
    if loc["geometry"]["type"] == "Point":
        curr_latlng = lonlat_swap(loc["geometry"]["coordinates"])
        return folium.Marker(curr_latlng, icon=folium.Icon(color=disp_color),
                  popup="%s" % loc["properties"]["name"])
    elif loc["geometry"]["type"] == "Polygon":
        assert len(loc["geometry"]["coordinates"]) == 1,\
            "Only simple polygons supported!"
        curr_latlng = [lonlat_swap(c) for c in loc["geometry"]["coordinates"][0]]
        # print("Returning polygon for %s" % curr_latlng)
        return folium.PolyLine(curr_latlng, color=disp_color, fill=disp_color,
                  popup="%s" % loc["properties"]["name"])        

In [6]:
def get_marker(loc, disp_color):
    if type(loc) == list:
        return [get_one_marker(l, disp_color) for l in loc]
    else:
        print("Found single entry, is this expected?")
        return [get_one_marker(loc, disp_color)]

### Read the data

In [7]:
spec_to_validate = json.load(open("final_sfbayarea_filled_reroutes/train_bus_ebike_mtv_ucb.filled.reroute.json"))
sensing_configs = json.load(open("sensing_regimes.all.specs.json"))

### Validating the time range

In [8]:
print("Experiment runs from %s -> %s" % (arrow.get(spec_to_validate["start_ts"]), arrow.get(spec_to_validate["end_ts"])))
start_fmt_time_to_validate = arrow.get(spec_to_validate["start_ts"]).format("YYYY-MM-DD")
end_fmt_time_to_validate = arrow.get(spec_to_validate["end_ts"]).format("YYYY-MM-DD")
if (start_fmt_time_to_validate != spec_to_validate["start_fmt_date"]):
    print("VALIDATION FAILED, got start %s, expected %s" % (start_fmt_time_to_validate, spec_to_validate["start_fmt_date"]))
if (end_fmt_time_to_validate != spec_to_validate["end_fmt_date"]):
    print("VALIDATION FAILED, got end %s, expected %s" % (end_fmt_time_to_validate, spec_to_validate["end_fmt_date"]))

Experiment runs from 2019-07-16T07:00:00+00:00 -> 2020-04-30T07:00:00+00:00


### Validating calibration trips

In [9]:
def get_map_for_calibration_test(trip):
    curr_map = folium.Map()
    if trip["start_loc"] is None or trip["end_loc"] is None:
        return curr_map
    curr_start = lonlat_swap(trip["start_loc"]["coordinates"])
    curr_end = lonlat_swap(trip["end_loc"]["coordinates"])
    folium.Marker(curr_start, icon=folium.Icon(color="green"),
                  popup="Start: %s" % trip["start_loc"]["name"]).add_to(curr_map)
    folium.Marker(curr_end, icon=folium.Icon(color="red"),
                  popup="End: %s" % trip["end_loc"]["name"]).add_to(curr_map)
    folium.PolyLine([curr_start, curr_end], popup=trip["id"]).add_to(curr_map)
    curr_map.fit_bounds([curr_start, curr_end])    
    return curr_map

In [10]:
calibration_tests = spec_to_validate["calibration_tests"]
rows = get_row_count(len(calibration_tests), 4)
calibration_maps = bre.Figure((rows,4))
for i, t in enumerate(calibration_tests):
    if t["config"]["sensing_config"] != sensing_configs[t["config"]["id"]]["sensing_config"]:
        print("Mismatch in config for test" % t)
    curr_map = get_map_for_calibration_test(t)
    calibration_maps.add_subplot(rows, 4, i+1).add_child(curr_map)
calibration_maps

### Validating evaluation trips

In [11]:
def get_map_for_travel_leg(trip):
    curr_map = folium.Map()
    [m.add_to(curr_map) for m in get_marker(trip["start_loc"], "green")]
    [m.add_to(curr_map) for m in get_marker(trip["end_loc"], "red")]
    # trips from relations won't have waypoints
    if "waypoint_coords" in trip:
        if isinstance(trip["waypoint_coords"], dict):
            for i, wpc in enumerate(trip["waypoint_coords"]["geometry"]["coordinates"]):
                folium.map.Marker(
                    lonlat_swap(wpc), popup="%d" % i,
                    icon=fof.DivIcon(class_name='leaflet-div-icon')).add_to(curr_map)
        else:
            for wpcs in trip["waypoint_coords"]:
                for i, wpc in enumerate(wpcs["geometry"]["coordinates"]):
                    folium.map.Marker(
                        lonlat_swap(wpc), popup="%d" % i,
                        icon=fof.DivIcon(class_name='leaflet-div-icon')).add_to(curr_map)
    
    latlng_route_coords = [[lonlat_swap(rc) for rc in coords["geometry"]["coordinates"]] for coords in trip["route_coords"]]
    for ll in latlng_route_coords:
        folium.PolyLine(ll, popup="%s: %s" % (trip["mode"], trip["name"])).add_to(curr_map)
        
    for i, c in enumerate(latlng_route_coords):
        folium.CircleMarker(c, radius=5, popup="%d: %s" % (i, c)).add_to(curr_map)
        
    curr_map.fit_bounds(ful.get_bounds(trip["route_coords"][0]["geometry"]["coordinates"], lonlat=True))
    
    return curr_map

SyntaxError: invalid syntax (<ipython-input-11-4acadc3f5563>, line 7)

In [ ]:
def get_map_for_shim_leg(trip):
    curr_map = folium.Map()
    mkrs = get_marker(trip["loc"], "purple")
    for mkr in mkrs:
        mkr.add_to(curr_map)
    curr_map.fit_bounds(mkrs[0].get_bounds())
    return curr_map

In [ ]:
evaluation_trips = spec_to_validate["evaluation_trips"]
map_list = []
for t in evaluation_trips:
    for l in t["legs"]:
        if l["type"] == "TRAVEL":
            curr_map = get_map_for_travel_leg(l)
            map_list.append(curr_map)
        else:
            curr_map = get_map_for_shim_leg(l)
            map_list.append(curr_map)

rows = get_row_count(len(map_list), 2)
evaluation_maps = bre.Figure(ratio="{}%".format((rows/2) * 100))
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(rows, 2, i+1).add_child(curr_map)
evaluation_maps

### Validating start and end polygons

In [ ]:
def check_start_end_contains(leg):
    points = gpd.GeoSeries([shpg.Point(p) for p in leg["route_coords"]["geometry"]["coordinates"]])
    start_loc = shpg.shape(leg["start_loc"]["geometry"])
    end_loc = shpg.shape(leg["end_loc"]["geometry"])
    start_contains = points.apply(lambda p: start_loc.contains(p))
    print(points[start_contains])
    end_contains = points.apply(lambda p: end_loc.contains(p))
    print(points[end_contains])
    # Some of the points are within the start and end polygons
    assert start_contains.any()
    assert end_contains.any()
    
    # The first and last point are within the start and end polygons
    assert start_contains.iloc[0], points.head()
    assert end_contains.iloc[-1], points.tail()
    
    # The points within the polygons are contiguous
    max_index_diff_start = pd.Series(start_contains[start_contains == True].index).diff().max()
    max_index_diff_end = pd.Series(end_contains[end_contains == True].index).diff().max()
    assert pd.isnull(max_index_diff_start) or max_index_diff_start == 1, "Max diff in index = %s for points %s" % (gpd.GeoSeries(start_contains[end_contains == True].index).diff().max(), points.head())
    assert pd.isnull(max_index_diff_end) or max_index_diff_end == 1, "Max diff in index = %s for points %s" % (gpd.GeoSeries(end_contains[end_contains == True].index).diff().max(), points.tail())

In [ ]:
invalid_legs = []
for t in evaluation_trips:
    for l in t["legs"]:
        if l["type"] == "TRAVEL" and l["id"] not in invalid_legs:
            print("Checking leg %s, %s" % (t["id"], l["id"]))
            check_start_end_contains(l)

### Validating sensing settings

In [ ]:
for ss in spec_to_validate["sensing_settings"]:
    for phoneOS, compare_map in ss.items():
        compare_list = compare_map["compare"]
        for i, ssc in enumerate(compare_map["sensing_configs"]):
            if ssc["id"] != compare_list[i]:
                print("Mismatch in sensing configurations for %s" % ss)